# Информационный поиск и парсинг

1) Регулярные выражения  
2) Фразовые запросы и ранжированный информационный поиск; методы оценки качества поисковых машин  
3) Парсинг. Синтаксис составляющих и синтаксис зависимостей; контекстосвободные грамматики;  Лексикализованные вероятностные грамматики; Применение парсинга в различных задачах.

### 1) Регулярные выражения  

При работе с текстами часто приходится решать задачу поиска в тексте по шаблону, например в текстовом редакторе или в браузере. 
Формализуем задачу: задается текстовая строка длиной $N$ и шаблон длиной $M$, нужно найти индекс первого вхождения шаблона в текст. Большинство алгоритмов для этой задачи можно легко расширить, чтобы найти все вхождения
шаблона в тексте, подсчитать количество вхождений шаблона в тексте или предоставить контекст (подстроки текста, окружающие каждое вхождение шаблона). Поиск подстроки - классическая задача, которая имеет множество интересных решений, ознакомиться с которыми можно в книге Р.Сэджвика Алгоритмы, 5.3. 

Однако сегодня мы поговорим о поиске на основании неполной информации (поиск по шаблону). Все возможные слова, подходящие под искомый шаблон, задают язык искомых слов. Для описания языка (множества) искомых слов используют регулярные выражения.  

**Регулярное выражение** (regexp, регулярка) - это шаблон, сопоставляемый с искомой строкой слева направо. 
Регулярки описывают образцы с помощью трех простых, но мощных операций: 
- конкатенация,  
- логическое или,  
- замыкание.  

Конкатенация позволяет определить язык (искомых слов) путем простой записи символов рядом друг с другом. Например, запись *AB* определяет язык *{AB}*, содержащий всего одно слово.  

Логическое ИЛИ позволяет определить несколько слов в языке. Например, запись *AB|CD* означает что нужно искать слова *AB* или *CD*.  

Замыкание позволяет повторять части образца произвольное число раз, в т.ч. ни разу. Например, регулярка A\*B  задает слова {B, AB, AAB, AAAB ...} а регулярка AB\* - A, AB, ABB, ABBB, ...  

Для определения порядка подстановки используются круглые скобки. Например, выражение C(AC|B)D задает множество слов {CACD, CBD}, а регулярка (AB)* - {$\in$ , AB, ABAB, ABABAB, ...} где $\in$ - пустая строка.  

Стандартная библиотека содержит модуль re для работы с ASCII алфавитом. Для работы с Unicode алфавитом используется сторонний модуль regexp.   

Метод match() объекта re.Pattern проверяет совпадение строки с регуляркой. Метод match проверяет соответствие шаблону с начала строки. В результате возвращается объект re.Match, в котором содержатся span - (индекс начала, длина совпадения) и match - совпавшая строка. 

In [1]:
import re

p = re.compile('a*') # pattern
sent = ['abababa',
        'aaaaaab',
        'bbbbbbb',
        'bababab'
       ]
        
for s in sent:
    print(p.match(s))

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 6), match='aaaaaa'>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 0), match=''>


Также можно использовать top-level методы из модуля re. Шаблон можно не компилировать, достаточно указать символ r перед строкой с одинарными кавычками. 

In [2]:
for s in sent:
    print(re.match(r'ab*', s))

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 1), match='a'>
None
None


In [3]:
# Метод search() ищет совпадения по всей строке.
for s in sent:
    print(re.search(r'ab*', s))

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 1), match='a'>
None
<re.Match object; span=(1, 3), match='ab'>


Задание: Напишите пример использования методов findall() и sub().

Кроме того, в регулярках можно использовать метасимволы:  
. - (dot) любой символ, кроме \n

In [4]:
for s in sent:
    print(re.match(r'.', s))

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='b'>
<re.Match object; span=(0, 1), match='b'>


^ - (caret) начало строки

In [5]:
for s in sent:
    print(re.match(r'^b', s))

None
None
<re.Match object; span=(0, 1), match='b'>
<re.Match object; span=(0, 1), match='b'>


$ - конец строки

In [6]:
for s in sent:
    print(re.search(r'a$', s))

<re.Match object; span=(6, 7), match='a'>
None
None
None


\+ - не меньше одного вхождения, в отличии от *, который допускает ни одного.

In [7]:
for s in sent:
    print(re.match(r'b+', s))

None
None
<re.Match object; span=(0, 7), match='bbbbbbb'>
<re.Match object; span=(0, 1), match='b'>


? - 0 или 1 вхождение

In [8]:
for s in sent:
    print(re.search(r'ab?', s))

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 1), match='a'>
None
<re.Match object; span=(1, 3), match='ab'>


{m} - определяет m количество вхождений

In [9]:
for s in sent:
    print(re.search(r'b{3}', s))

None
None
<re.Match object; span=(0, 3), match='bbb'>
None


{m,n} - количество вхождений от m до n

In [10]:
for s in sent:
    print(re.search(r'b{3,5}', s))

None
None
<re.Match object; span=(0, 5), match='bbbbb'>
None


{m,n}? - жадное (минимальное) количество вхождений от m до n

In [11]:
for s in sent:
    print(re.search(r'b{3,5}?', s))

None
None
<re.Match object; span=(0, 3), match='bbb'>
None


\ - обратный слэш, экранирует спецсимволы

In [12]:
s = '***'
re.match(r'\*', s)

<re.Match object; span=(0, 1), match='*'>

[ ] - определяет множество символов

In [13]:
for s in sent:
    print(re.match(r'[ab]a', s))

None
<re.Match object; span=(0, 2), match='aa'>
None
<re.Match object; span=(0, 2), match='ba'>


[a-z] - определяет диапазон символов

In [14]:
re.match(r'[a-c]', 'bcde')

<re.Match object; span=(0, 1), match='b'>

In [15]:
re.match(r'[0-5][0-9]', '59')

<re.Match object; span=(0, 2), match='59'>

Также доступны шаблоны:  
- \w слово в нижнем регистре  
- 

### 2) Фразовые запросы и ранжированный информационный поиск

**Поисковая система** (search engine) — программа, предоставляющая пользователю возможность быстрого доступа к необходимой ему информации при помощи алгоритмов поиска заданной фразы в обширной коллекции данных.  
Пользователь формулирует поисковый запрос, система находит документы, которые содержат указанные ключевые слова или близкие им по смыслу.  
Если говорить о поиске в интернете, то база документов составляет более 2 млрд страниц. Ежедневно специальные программы (поисковые пауки) обходят содержимое сайтов и индексируют их. 
<img src='imgs/search_engine.jpg'>

Индексирование, совершаемое поисковой машиной, это процесс сбора, сортировки и хранения данных с целью обеспечить быстрый и точный поиск информации.  
Поисковый индекс — это структура данных, которая содержит информацию о документах и используется в поисковых системах.  
Например, для поиска среди документов, связанных гиперссылками, применяется алгоритм ранжирования PageRank. Алгоритм помогает пользователю, выдавая среди результатов поисковой выдачи авторитетные источники в первую очередь. PageRank — это числовая величина, характеризующая «важность» веб-страницы. Чем больше ссылок на страницу, тем она «важнее». Кроме того, «вес» страницы А определяется весом ссылки, передаваемой страницей B. Таким образом, PageRank — это метод вычисления веса страницы путём подсчёта важности ссылок на неё.  

<img src='imgs/page_rank.png' width = 400>

На рисунке представлен пример PageRank для простой сети, выраженный в процентах (Google использует логарифмическую шкалу). Вебстраница C имеет более высокий рейтинг, чем страница E, хотя количество ссылок на C меньше, чем на Е. Всего одна из ссылок на C исходит из более важной страницы B, следовательно, С имеет более высокое значение PageRank.  

Для оценки качества поискового поиска используется множество методов. Рассмотрим две самые полезные метрики: точность и полноту поиска.  

**Точность** (precision) - отношение числа релевантных документов, найденных поиском, к общему числу найденных документов:

$${\mbox{Precision}}={\frac  {|D_{{rel}}\cap \ D_{{retr}}|}{|D_{{retr}}|}}$$

где $D_{rel}$ — это множество релевантных документов в базе, а $D_{retr}$ — множество документов, найденных системой.  

Точность еще можно вычислить как $1 - P_{I}$ величина ошибки 1 рода - доля найденных документов, не соответствующих поисковому запросу. Чем выше точность, тем меньше мусора в выдаче.  

**Полнота** (recall) - Отношение числа найденных релевантных документов, к общему числу релевантных документов в базе:
это доля потерянных документов, не найденных поисковой машиной (ошибки 2 рода).  

$${\mbox{Recall}}={\frac  {|D_{{rel}}\cap \ D_{{retr}}|}{|D_{{rel}}|}}$$

где $D_{rel}$ — это множество релевантных документов в базе, а $D_{retr}$ — множество документов, найденных системой.  
Полноту также можно вычислить как $1 - P_{II}$ величину ошибки 2 рода, доля не найденных релевантных документов. Чем выше полнота, тем меньше документов упустила поисковая машина. 

Рассмотрим пример. Пусть в базе имеется 1000 документов. Пользователь вводит запрос, которому в действительности соответствует 100 документов из базы. Поисковая машина выдает пользователю 90 документов, 5 из которых не соответствуют поисковому запросу.  

Точность равна $ 85 / 90 = 94\%$  
Полнота равна $ 85 / 100 = 85\%$  

Задание: напишите формулу для вычисления точности и полноты, зная количество неправильных документов в выдаче (D_1) и количество ненайденных документов (D_2).

In [16]:
# Количество правильных документов в выдаче
D_rel_retr = 85

# Неправильные документы в выдаче
D_1 = 5
# Ненайденные документы в выдаче
D_2 = 15

# Your code here


### 3) Парсинг

**Парсинг** (синтаксический анализ, разбор, parsing) — процесс сопоставления линейной последовательности лексем (слов, токенов) естественного или формального языка с его формальной грамматикой. Каждый из нас будучи школьником делал синтаксический анализ предложений - выделение структурных элементов (подлежащего, сказуемого) и связей между ними.   Результатом парсинга обычно является дерево разбора (синтаксическое дерево).   

In [1]:
import spacy
nlp = spacy.load('ru_core_news_md')

In [2]:
text = 'мой дядя самых честных правил'
document = nlp(text)
for token in document:
    print(token.lemma_, token.pos_, token.dep_)

мой DET det
дядя NOUN ROOT
самых ADJ amod
честной ADJ amod
правило NOUN nmod


Часто под парсингом подразумевают **скрапинг** (web scraping) - автоматизированный сбор информации с сайтов, анализ, преобразование и выдача в структурированном виде (таблицы с набором данных). Это лишь частный случай применения парсинга для решения бизнес-задач. Кроме этого, парсинг используется для разбора исходных текстов программ, структуированных данных (XML, JSON, YAML и т.д.), построения индекса поисковых систем и многого другого.  

На рисунке представлен пример парсинга html-строки простейшего веб-сайта. 

<img src='imgs/parsing_html.jpg'>


Кроме иерархических структур, результатом парсинга может быть другая структура данных, например очередь (queue) классифицированных лексем.  

По количеству операций чтения входных данных парсеры могут быть:  
- однопроходные  
- многопроходные.    

По порядку построения вывода парсеры могут быть:  
- восходящиея распознаватели - порождают цепочки правостороннего вывода и строят дерево вывода снизу вверх;
- нисходящие распознаватели - порождают цепочки левостороннего вывода и строят дерево вывода сверху вниз.  

С формальной точки зрения, процесс парсинга - это распознавание языка, задаваемого контекстно-свободной грамматикой (тип 2 по Хомскому) с помощью  автомата с магазинной памятью.  

С практической же точки зрения, для парсинга веб страниц существует множество готовых библиотек, например beatifulsoup.  
Рассмотрим скрипт, с помощью которого можно получить содержимое веб страницы в виде текста.

In [28]:
import requests
import bs4

url = 'https://www.rbc.ru/short_news'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text)

print(soup)

<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=no, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
<meta content="True" name="HandheldFriendly"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="address=no" name="format-detection"/>
<meta content="97bc592b27a9ada2d9a4bb418ed0ebed" name="csrf-token"/>
<meta content="csrf_token" name="csrf-param"/>
<meta content="rbс" name="Copyright"/>
<title>Главные новости дня в России и мире</title>
<meta content="Главные новости дня в России и мире" name="title">
<meta content="Свежие новости дня мира и России на РБК. «РосБизнесКонсалтинг» — ведущая российская компания, работающая в сферах масс-медиа и информационных технологий. Последние новости сегодня в политике, экономике,

In [30]:
news = soup.find_all('span', attrs={'class':'item__title rm-cm-item-text'}) 
news_list = [n.text.strip() for n in news]
print(*news_list, sep='\n')

Медведев пробился в четвертьфинал турнира серии «Мастерс» в США
Шлепанцы, бургер, тушь: что мешает людям водить машину
Из-за пожара в лесу в Рязанской области возбудили уголовное дело
В России до 2035 года локализуют производство автодвигателей и ABS
«Будильник» для водителя: решение проблемы усталости за рулем
Военная операция на Украине. Главное
Экс-арбитр РПЛ насчитал большинство судейских ошибок в пользу «Спартака»
Какой может быть математика наказаний для водителей
В Керчи сработали системы ПВО
Клуб Черчесова разгромил «Шэмрок Роверс» в квалификации Лиги Европы
Как борются с рассеянностью пешеходов за границей
Оперштаб остановил публикацию данных об уровне коллективного иммунитета
«Хочу отдать почку»: почему нужно изменить закон о пересадке органов
«Фосагро» определился с дивидендами. Доходность может составить почти 10%


Задание:  
Найдите интересный сайт и напишите скрипт для парсинга его содержимого.

In [19]:
# Your code is here



Ссылки:  

- Сэджвик Р. и др., Алгоритмы, Гл. 5.3
- Hobson Lane etc, NLP in action, Гл. 11
- https://docs.python.org/3/library/re.html
- https://docs.python.org/3/howto/regex.html
- https://habr.com/ru/post/349860/
- https://habr.com/ru/company/alconost/blog/339894/
- https://www.youtube.com/watch?v=8iE2CYxxNdY